In [37]:
## import libraries
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation, TimeDistributed, RepeatVector
import matplotlib.pyplot as plt
#import os
import tensorflow as tf

In [38]:
# set GPU memory
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [39]:
n_step = 5

In [40]:
def split(data, depth):
    dataset = data
    depth = depth
    X = np.zeros([int(depth), n_step, 3])
    Y = np.zeros([int(depth), n_step, 3])
    c = 0
    d = 0
    
    for i in range(int(depth)):
        for j in range(n_step):
            if d < dataset.shape[0]:
                X[i,j,:] = dataset[d, c:c+3]
                Y[i,j,:] = dataset[d, (c+3*n_step):(c+3*n_step+3)]
                
                if ((c+3*n_step+3) != (dataset.shape[1])):
                    c +=3
                else:
                    c = 0
                    d += 1
        if (c-3) > 0:
            c = (c - 3*n_step + 3)
        else:
            c = c
            
    return X, Y

In [41]:
dataset = pd.read_csv('/home/lab606a/Documents/20200331/fixed/origin data 30hz/top5.csv', header=None)
dataset = dataset.fillna(0)
dataset = np.array(dataset)
dataset.shape

(816, 129)

In [42]:
maxlen_train = dataset.shape[1]+12

In [43]:
dataset = sequence.pad_sequences(dataset, maxlen=maxlen_train, padding='post', dtype='float32')
dataset.shape

(816, 141)

In [44]:
depth_train = (int(dataset.shape[1]/3)+1-n_step-n_step)*dataset.shape[0] # (all_balls + 1 - input_balls - output_balls)*n_rows

In [45]:
x_train, y_train = split(data=dataset, depth=depth_train)

In [46]:
model = Sequential()
model.add(LSTM(512, activation='linear', input_shape=(x_train.shape[1], x_train.shape[2])))
#model.add(LSTM(128, activation='linear'))
model.add(RepeatVector(x_train.shape[1]))
#model.add(LSTM(128, activation='linear', return_sequences=True))
model.add(LSTM(512, activation='linear', return_sequences=True))
model.add(TimeDistributed(Dense(3)))

In [47]:
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [48]:
history = model.fit(x_train, y_train, batch_size=4000, epochs=2000, shuffle=True)

Epoch 1/2000
22848/22848 [==============================] - 2s 95us/step - loss: 3067.0299 - acc: 0.5495
Epoch 2/2000
22848/22848 [==============================] - 1s 46us/step - loss: 2299.1719 - acc: 0.7173
Epoch 3/2000
22848/22848 [==============================] - 1s 46us/step - loss: 2409.2805 - acc: 0.6669
Epoch 4/2000
22848/22848 [==============================] - 1s 46us/step - loss: 1584.8389 - acc: 0.6783
Epoch 5/2000
22848/22848 [==============================] - 1s 46us/step - loss: 2243.4969 - acc: 0.6094
Epoch 6/2000
22848/22848 [==============================] - 1s 46us/step - loss: 2452.3514 - acc: 0.6686
Epoch 7/2000
22848/22848 [==============================] - 1s 46us/step - loss: 4023.5259 - acc: 0.5156
Epoch 8/2000
22848/22848 [==============================] - 1s 46us/step - loss: 11811.5917 - acc: 0.4841
Epoch 9/2000
22848/22848 [==============================] - 1s 47us/step - loss: 8351.3384 - acc: 0.5390
Epoch 10/2000
22848/22848 [===========================

KeyboardInterrupt: 

In [ ]:
# plot accuracy history
# summarize history for accuracy 
plt.plot(history.history['acc'])
#plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
#acc_png = './training history/' + file_name + '_accuracy.png'
#plt.savefig(acc_png)
plt.show()